In [26]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

df = pd.read_csv("data/Asia and Pacific Regional Economic Outlook (APDREO)/Dataset_tahun2000.csv")

df


,DATASET,SERIES_CODE,OBS_MEASURE,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,IMF.APD:APDREO(6.0.0),IDN.GGXCNL_GDP.A,OBS_VALUE,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,...,-2.56,-2.26,-1.69,-2.09,-6.07,-4.40,-2.31,-1.64,NaN,NaN
1,IMF.APD:APDREO(6.0.0),IDN.NGDP_R_PPP_PC_PCH.A,OBS_VALUE,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,...,3.69,3.92,4.06,3.94,-3.26,2.76,4.15,3.93,5.03,NaN
2,IMF.APD:APDREO(6.0.0),IDN.BCA_GDP_BP6.A,OBS_VALUE,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,...,-1.82,-1.59,-2.94,-2.71,-0.42,0.30,1.00,-0.15,NaN,NaN
3,IMF.APD:APDREO(6.0.0),IDN.PCPIE_PCH.A,OBS_VALUE,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,...,3.03,3.61,3.16,2.58,1.68,1.88,5.41,2.81,1.57,2.72
4,IMF.APD:APDREO(6.0.0),IDN.NGDP_RPCH.A,OBS_VALUE,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,...,5.03,5.07,5.17,5.02,-2.07,3.70,5.31,5.05,NaN,NaN
5,IMF.APD:APDREO(6.0.0),IDN.LUR.A,OBS_VALUE,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,...,5.61,5.50,5.34,5.28,7.07,6.49,5.86,5.32,4.91,4.85
6,IMF.APD:APDREO(6.0.0),IDN.PCPI_PCH.A,OBS_VALUE,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,...,3.53,3.81,3.29,2.82,2.03,1.56,4.14,3.71,1.57,2.72


In [27]:
df = df.drop(columns=["DATASET", "SERIES_CODE", "OBS_MEASURE"])


In [28]:
df

,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,-1.08,-0.26,0.46,...,-2.56,-2.26,-1.69,-2.09,-6.07,-4.40,-2.31,-1.64,NaN,NaN
1,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,3.31,3.55,4.21,...,3.69,3.92,4.06,3.94,-3.26,2.76,4.15,3.93,5.03,NaN
2,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,3.17,1.88,0.51,...,-1.82,-1.59,-2.94,-2.71,-0.42,0.30,1.00,-0.15,NaN,NaN
3,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,5.19,6.38,17.14,...,3.03,3.61,3.16,2.58,1.68,1.88,5.41,2.81,1.57,2.72
4,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,4.78,5.03,5.69,...,5.03,5.07,5.17,5.02,-2.07,3.70,5.31,5.05,NaN,NaN
5,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,9.67,9.86,11.24,...,5.61,5.50,5.34,5.28,7.07,6.49,5.86,5.32,4.91,4.85
6,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,6.79,6.06,10.46,...,3.53,3.81,3.29,2.82,2.03,1.56,4.14,3.71,1.57,2.72


# Menggunakan Model ARIMA


In [29]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

# Load data
df = df.copy()

# 3. Tahun yang dipakai untuk forecasting
years_2000_2023 = [str(y) for y in range(2000, 2024)]  # training untuk 2024
years_2000_2024 = [str(y) for y in range(2000, 2025)]  # training untuk 2025

def arima_forecast(series, steps):
    series = series.dropna()

    if len(series) < 5:
        return [None] * steps

    try:
        model = ARIMA(series, order=(1,1,1))
        res = model.fit()
        preds = res.forecast(steps=steps)
        return preds.tolist()
    except:
        return [None] * steps

# --- Tahap 1: Forecast / isi tahun 2024 --- #
for i, row in df.iterrows():
    if pd.isna(row["2024"]):
        series_2000_2023 = row[years_2000_2023].astype(float)
        pred2024 = arima_forecast(series_2000_2023, steps=1)[0]
        df.at[i, "2024"] = pred2024

# --- Tahap 2: Forecast / isi tahun 2025 --- #
for i, row in df.iterrows():
    if pd.isna(row["2025"]):
        series_2000_2024 = row[years_2000_2024].astype(float)
        pred2025 = arima_forecast(series_2000_2024, steps=1)[0]
        df.at[i, "2025"] = pred2025

df




,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,-1.08,-0.26,0.46,...,-2.56,-2.26,-1.69,-2.09,-6.07,-4.40,-2.31,-1.64,-1.97,-2.14
1,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,3.31,3.55,4.21,...,3.69,3.92,4.06,3.94,-3.26,2.76,4.15,3.93,5.03,3.92
2,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,3.17,1.88,0.51,...,-1.82,-1.59,-2.94,-2.71,-0.42,0.30,1.00,-0.15,-0.09,-0.14
3,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,5.19,6.38,17.14,...,3.03,3.61,3.16,2.58,1.68,1.88,5.41,2.81,1.57,2.72
4,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,4.78,5.03,5.69,...,5.03,5.07,5.17,5.02,-2.07,3.70,5.31,5.05,4.85,4.85
5,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,9.67,9.86,11.24,...,5.61,5.50,5.34,5.28,7.07,6.49,5.86,5.32,4.91,4.85
6,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,6.79,6.06,10.46,...,3.53,3.81,3.29,2.82,2.03,1.56,4.14,3.71,1.57,2.72


# Menggunakan Model ETS untuk TAHUN 2025-2026

In [37]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Tahun historis yang lengkap (2000–2025)
years_2000_2025 = [str(y) for y in range(2000, 2026)]

# Tambahkan kolom untuk forecast 2026–2030
for year in range(2026, 2031):
    df[str(year)] = None

for i, row in df.iterrows():
    # Ambil data historis
    series = row[years_2000_2025].astype(float).dropna()

    # Minimal 5 data agar ETS stabil
    if len(series) < 5:
        continue

    try:
        # Model ETS tanpa seasonality untuk data tahunan
        model = ExponentialSmoothing(series, trend="add", seasonal=None)
        model_fit = model.fit()

        # Forecast 5 tahun: 2026–2030
        preds = model_fit.forecast(5)

        df.at[i, "2026"] = preds.iloc[0]
        df.at[i, "2027"] = preds.iloc[1]
        df.at[i, "2028"] = preds.iloc[2]
        df.at[i, "2029"] = preds.iloc[3]
        df.at[i, "2030"] = preds.iloc[4]

    except Exception as e:
        print(f"ETS error untuk indikator: {row['INDICATOR']}")
        print(e)

df
# df.to_excel("data/Asia and Pacific Regional Economic Outlook (APDREO)/DataForecast_2000-2030.xlsx", index=False)


,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,2003,2004,2005,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
0,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,-1.08,-0.26,0.46,...,-4.40,-2.31,-1.64,-1.97,-2.14,-2.19,-2.22,-2.24,-2.26,-2.29
1,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,3.31,3.55,4.21,...,2.76,4.15,3.93,5.03,3.92,3.41,3.39,3.37,3.35,3.34
2,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,3.17,1.88,0.51,...,0.30,1.00,-0.15,-0.09,-0.14,-0.33,-0.51,-0.70,-0.88,-1.07
3,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,5.19,6.38,17.14,...,1.88,5.41,2.81,1.57,2.72,1.04,0.70,0.35,0.00,-0.35
4,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,4.78,5.03,5.69,...,3.70,5.31,5.05,4.85,4.85,4.29,4.24,4.19,4.14,4.09
5,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,9.67,9.86,11.24,...,6.49,5.86,5.32,4.91,4.85,4.80,4.75,4.70,4.65,4.60
6,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,6.79,6.06,10.46,...,1.56,4.14,3.71,1.57,2.72,1.53,1.22,0.92,0.61,0.30
